### 1. import package

In [1]:
import requests
import pandas as pd
import re
import time

from selenium import webdriver

### 2. set wait time

In [2]:
wait_time = 1

### 3. functions

In [3]:
def make_url(category, date, page):
    url = "http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=" + str(category) + "#&date=" + date + " 00:00:00&page=" + str(page)
    return url

In [4]:
def last_page(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)

    time.sleep(wait_time)
    pages = driver.find_elements_by_css_selector("#paging ._paging")
    total_page = len(pages)
    
    if len(pages) > 9:
        
        pages[9].click()        
        time.sleep(wait_time)
        pages = driver.find_elements_by_css_selector("#paging ._paging")
        total_page += len(pages) - 1
        
        while len(pages) > 10:
        
            pages[10].click()        
            time.sleep(wait_time)
            pages = driver.find_elements_by_css_selector("#paging ._paging")
            
            if len(pages) > 10:
                total_page += len(pages) - 1 
            else:
                total_page += len(pages)
            
    driver.close()   
    
    return total_page

In [5]:
def last_page(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)

    time.sleep(wait_time)
    pages = driver.find_elements_by_css_selector("#paging ._paging")
    total_page = len(pages)
    
    if len(pages) > 9:
        
        pages[9].click()        
        time.sleep(wait_time)
        pages = driver.find_elements_by_css_selector("#paging ._paging")
        total_page += len(pages) - 1
        
        while len(pages) > 10:
        
            pages[10].click()        
            time.sleep(wait_time)
            pages = driver.find_elements_by_css_selector("#paging ._paging")
            
            if len(pages) > 10:
                total_page += len(pages) - 1 
            else:
                total_page += len(pages)
            
    driver.close()   
    
    return total_page

In [6]:
def get_content(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)
    time.sleep(wait_time)
    content = driver.find_element_by_css_selector("#articleBodyContents").text
    
    comment = 0
    comment_element = driver.find_elements_by_css_selector("#articleTitleCommentCount .lo_txt")
    if int(len(comment_element)) > 0:
        comment = comment_element[0].text
    
    likeit = 0
    likeit_element = driver.find_elements_by_css_selector(".u_likeit_module .u_cnt")
    if int(len(likeit_element)) > 0:
        likeit = likeit_element[0].text
    
    driver.close()
    
#     comment, likeit, content = 0,0,"-"
    
    return comment, likeit, content

In [7]:
def article_url_list(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)
    
    time.sleep(wait_time)
    articles = driver.find_elements_by_css_selector("#section_body.section_body ul li")
    
    article_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content"])
    
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        title = article.find_element_by_css_selector("a").get_attribute("title")
        newspaper = article.find_element_by_css_selector("span").text
        newsid = link.split("aid=")[1]
#         comment, likeit, content = get_content(link) 
        comment, likeit, content = 0,0,"-"
        
        tmp_dict = {
            "newsid": newsid, 
            "newspaper": newspaper, 
            "title": title, 
            "link": link, 
            "comment": comment, 
            "likeit": likeit, 
            "content": content,
        }
        
        article_df.loc[len(article_df)] = tmp_dict
        
    article_df["date"] = url.split("date=")[1].split(" ")[0]
    article_df["category"] = url.split("sid1=")[1].split("#")[0]
    
    driver.close()
    
    return article_df

In [8]:
def date_category_df(category, date):
    
    url = make_url(category, date, 1)
    last_page_number = last_page(url)
    
    result_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content", "date", "category"])

    for page in range(1, last_page_number + 1):
        link = make_url(category, date, page)
        tmp_df = article_url_list(link)        
        result_df = pd.concat([result_df, tmp_df])
        print(last_page_number, page)
        
    return result_df

### 4. crawling news list

In [9]:
def crawling_news_list():
    
    for category in range(100,106):
        result = main(category,"2016-01", 31)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-01.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-02", 28)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-02.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-03", 31)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-03.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-04", 30)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-04.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-05", 31)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./datas/" + str(category) + "_2016-05.csv", index=False, encoding="utf-8")

    for category in range(100,106):
        result = main(category,"2016-06", 30)
        result_df = result.reset_index(drop=True)
        result_df.to_csv("./" + str(category) + "_2016-06.csv", index=False, encoding="utf-8")

### 5. crawling contents in news list

In [24]:
def set_content(path):
    df = pd.read_csv(path)
    print(len(df))
    
    for idx, row in df.iterrows():
        
        if idx == 5:
            break
        
        if row.content == "-":
            
            print(idx, row.link)
            
            comment, likeit, content = get_content(row.link)
            df.loc[idx, "comment"] = comment
            df.loc[idx, "comment"] = likeit
            df.loc[idx, "content"] = content
            df.to_csv(path, index=False, encoding="utf-8")
            
#     return df

path = "./datas/105_2016-01.csv"
set_content(path)    

3250


In [23]:
path = "./datas/105_2016-01.csv"
df = pd.read_csv(path)
df.head()

,newsid,newspaper,title,link,comment,likeit,content,date,category
0,3572751,파이낸셜뉴스,"마이크로소프트 ""자사 서비스 이용자들에 해킹감지 공지한다""",http://news.naver.com/main/read.nhn?mode=LSD&m...,2.0,0.0,글로벌 컴퓨터 프로그램 업체 마이크로소프트(MS)가 이메일 등 자사 서비스 이용자들...,2016-01-01,105
1,2657109,서울신문,올해 첫 유성우 4일 밤 절정 '소원 빌어봐요',http://news.naver.com/main/read.nhn?mode=LSD&m...,8.0,0.0,"[서울신문 나우뉴스]\n\n사분의자리 유성우, 자정부터 보기 쉬워\n\n올초 밤하늘...",2016-01-01,105
2,3606373,머니투데이,"콘텐츠, 모바일에 맞춰 ""줄이고 잘라라""",http://news.naver.com/main/read.nhn?mode=LSD&m...,6.0,0.0,[머니투데이 서진욱 기자] ['자주 짧은' 이용패턴에 맞춰 콘텐츠 변화… 분량 줄고...,2016-01-01,105
3,3514369,한국경제,스마트카·로봇·드론…CES 2016 달군다,http://news.naver.com/main/read.nhn?mode=LSD&m...,5.0,0.0,미국 라스베이거스서 6일 개막\n\n\n[ 김현석 기자 ]\n스마트카 드론 로봇 등...,2016-01-01,105
4,3514355,한국경제,"[2016년 빛낼 '핀테크 스타트업 톱10'] 핀테크 개척자들 ""인터넷은행 시대, ...",http://news.naver.com/main/read.nhn?mode=LSD&m...,4.0,0.0,휴대폰 번호만 알면 전세계 누구에게나 송금\n은행 방문하지 않고도 주택담보대출 받을...,2016-01-01,105


In [ ]:
def merge(category):
    
    df_list = []
    
    for month in range(1,7):
        link_str = "./news_data/" + str(category) + "_2016-0" + str(month) + ".csv"

        df = pd.read_csv(link_str)
        df_list.append(df)
        
        print(link_str)

    return pd.concat(df_list).reset_index(drop=True)
    
for category in range(100, 106):
    merge(category)
    